# Setup

## 0.1 Importing Necessary Library

In [1]:
import torch
import torchvision
from torch import nn
import matplotlib.pyplot as plt
from torchvision import transforms

try:
  from torchinfo import summary
except:
  print("Library not found....")
  !pip install -q torchinfo
  from torchinfo import summary
  print("Library downloaded and imported!")

print(torch.__version__)
print(torchvision.__version__)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.1.0+cpu
0.16.0+cpu


## 0.2 Getting the going_modular script from github

In [2]:
try:
  from going_modular import data_setup,engine
except:
  print("Files not found....")
  !git clone https://github.com/HarshEkbote/PyTorch-Basics.git
  !mv PyTorch-Basics/going_modular .
  !rm -rf PyTorch-Basics
  from going_modular import data_setup,engine
  print("going_modular sript Imported!")

## 0.3 Device Agnostic code

In [3]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device:{device}")

Device:cpu


# Getting Data

In [4]:
import os
import zipfile
from pathlib import Path

import requests

data_path=Path('data/')
image_path=data_path/'pizza_steak_sushi'

if image_path.is_dir():
  print(f"{image_path} directory exists")
else:
  image_path.mkdir(parents=True,exist_ok=True)
  print("Directory not found, Creating Directory...")

  with open(data_path/'pizza_steak_sushi.zip','wb') as f:
    request=requests.get('https://github.com/HarshEkbote/PyTorch-Basics/raw/main/data/pizza_steak_sushi.zip')
    f.write(request.content)

  with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip','r') as zipref:
    zipref.extractall(image_path)
  
  os.remove(data_path/'pizza_steak_sushi.zip')

data\pizza_steak_sushi directory exists


In [5]:
train_dir=image_path/'train'
test_dir=image_path/'test'

# Creating Dataset and DataLoader

## 2.1 Manualing creating Transform according to the model

* Minibatches [batch_size,3,H,W]
* Tensor between [0,1]
* Mean [0.485,0.456,0.406]
* Standard Deviation [0.229,0.224,0.225]

In [6]:
from torchvision import transforms
manual_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [7]:
# Creating DataLoader using going_modular function
train_dataloader,test_dataloader,class_name=data_setup.create_dataloader(train_dir=train_dir,test_dir=test_dir,transform=manual_transform,batch_size=32)

train_dataloader,test_dataloader,class_name

(<torch.utils.data.dataloader.DataLoader at 0x23394f51de0>,
 ['pizza', 'steak', 'sushi'])

## 2.2 Creating transforms for torchvision.models using auto creation

In [8]:
#Get the weights of the model we need
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [9]:
#Get the transforms from the model
auto_transform=weights.transforms()
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [10]:
#Get the dataloader 
train_dataloader,test_dataloader,class_name=data_setup.create_dataloader(train_dir=train_dir,test_dir=test_dir,transform=auto_transform,batch_size=32)

train_dataloader,test_dataloader,class_name

(<torch.utils.data.dataloader.DataLoader at 0x23394f8be80>,
 ['pizza', 'steak', 'sushi'])